In [ ]:
# pip install chatterbot

# tfidf
# word2 vec
# bert based word embeddings

# pip install -U sentence-transformers

# pip install ipywidgets

In [1]:

import pandas as pd
import os
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split as tts
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder as LE
from sklearn.metrics.pairwise import cosine_similarity

import nltk
from nltk.corpus import stopwords
import datetime

import speech_recognition as sr
import pyttsx3 
import pyaudio


In [2]:
# Reading csv file

data = pd.read_csv("Chatbot Data.csv", encoding = "ISO-8859-1", engine='python')

data.head()

,Question,Answer,Category
0,What is the definition of Basic Savings Bank ...,The Basic Savings Bank Deposit Account or BSBD...,Basic Savings Bank Deposit Account
1,Whether the guidelines issued on no-frills acc...,No. In supersession of instructions contained ...,Basic Savings Bank Deposit Account
2,Can an Individual have any number of 'Basic Sa...,No. An individual is eligible to have only one...,Basic Savings Bank Deposit Account
3,Whether a 'Basic Savings Bank Deposit Account'...,Holders of 'Basic Savings Bank Deposit Account...,Basic Savings Bank Deposit Account
4,Can an individual have other deposit accounts ...,"Yes. One can have Term/Fixed Deposit, Recurrin...",Basic Savings Bank Deposit Account


In [3]:
data.isna().any()

Question    False
Answer      False
Category    False
dtype: bool

In [4]:
data.columns = ['Question','Answer','Class']

In [5]:
#defining stopwords

stop_words = set(stopwords.words('english'))
# print(stop_words)

In [6]:
# Defining label encoder and TfidfVectorizer

le = LE()

tfv = TfidfVectorizer(min_df=1, stop_words='english')


In [7]:
questions = data['Question'].values

In [8]:
#Tokenization and stopword removal

def cleanup(sentence):




    word_tok = nltk.word_tokenize(sentence)
    stemmed_words = [w for w in word_tok if not w in stop_words]
    #print(stemmed_words)

    #stemmed_words = [stemmer.stem(w) for w in word_tok]



    return ' '.join(stemmed_words)


In [9]:
cleanup("How much maximum amount can we transfer through rtgs")

'How much maximum amount transfer rtgs'

In [10]:
X = []

for question in questions:
    X.append(cleanup(question))

In [11]:
X[125]

'Does Reserve Bank powers lay standards ?'

In [12]:
#vectorization

tfv.fit(X)
le.fit(data['Class'])

X = tfv.transform(X)

y = le.transform(data['Class'])

trainx, testx, trainy, testy = tts(X, y, test_size=.3, random_state=42)

In [13]:
model = SVC(kernel='linear')
model.fit(trainx, trainy)


class_=le.inverse_transform(model.predict(X))



In [14]:
y_decoded = le.inverse_transform(y)

In [15]:
from sklearn.metrics import accuracy_score

accuracy_score(y_decoded, class_)

0.9481865284974094

In [16]:
def get_max5(arr):
    ixarr = []
    for ix, el in enumerate(arr):
        ixarr.append((el, ix))


    ixarr.sort()

    ixs = []
    for i in ixarr[-5:]:

        ixs.append(i[1])

    return ixs[::-1]


In [17]:
def get_response(usrText):





    while True:











        if usrText.lower() == "bye":
            return "Bye"



        GREETING_INPUTS = ["hello", "hi", "greetings", "sup", "what's up", "hey","hiii","hii","yo"]

        a = [x.lower() for x in GREETING_INPUTS]

        sd=["Thanks","Welcome"]

        d = [x.lower() for x in sd]


        am=["OK"]

        c = [x.lower() for x in am]

       # ty = ["getting"]
       # r = [x.lower() for x in ty]



        t_usr = tfv.transform([cleanup(usrText.strip().lower())])
        class_ = le.inverse_transform(model.predict(t_usr))

        questionset = data[data['Class'].values == class_]

        cos_sims = []
        for question in questionset['Question']:
            sims = cosine_similarity(tfv.transform([question]), t_usr)

            cos_sims.append(sims)

        ind = cos_sims.index(max(cos_sims))

        b = [questionset.index[ind]]









        if usrText.lower() in a:

            return ("Hi, I'm Finbot!\U0001F60A")


        if usrText.lower() in c:
            return "Ok...Alright!\U0001F64C"

        if usrText.lower() in d:
            return ("My pleasure! \U0001F607")



        if max(cos_sims) > [[0.]]:
            a = data['Answer'][questionset.index[ind]]+"   "
            return a


        elif max(cos_sims)==[[0.]]:
           return "sorry! \U0001F605"

In [18]:
def get_response2(usr):
    if usr.lower() == "bye":
        return "Thanks for having conversation! \U0001F60E"

    GREETING_INPUTS = ["hello", "hi", "greetings", "sup", "what's up", "hey","hii","hiii","hiiiii","yo","Hey there"]

    a = [x.lower() for x in GREETING_INPUTS]

    sd = ["Thanks", "Welcome"]

    d = [x.lower() for x in sd]

    am = ["OK"]

    c = [x.lower() for x in am]



    t_usr = tfv.transform([cleanup(usr.strip().lower())])
    class_ = le.inverse_transform(model.predict(t_usr))

    questionset = data[data['Class'].values == class_]

    cos_sims = []
    for question in questionset['Question']:
        sims = cosine_similarity(tfv.transform([question]), t_usr)

        cos_sims.append(sims)

    ind = cos_sims.index(max(cos_sims))

    b = [questionset.index[ind]]

    if usr.lower() in a:
        return ("you can ask me questions related to: Accounts, Investments, Funds, etc.")

    if usr.lower() in c:
        return " Cool! \U0001f604"

    if usr.lower() in d:
        return ("\U0001F44D")




    if max(cos_sims) == [[0.]]:
        return "I'm not able to solve this question at this moment. You can call to customer support 1860 999 9999 \U0001F615"

    if max(cos_sims) > [[0.]]:

        inds = get_max5(cos_sims)
        print(inds)

        b = "(1)" + data['Question'][questionset.index[0]]
        c = "(2)" + data['Question'][questionset.index[1]]
        d = "(3)" + data['Question'][questionset.index[2]]
        e = "(4)" + data['Question'][questionset.index[3]]
        f = "(5)" + data['Question'][questionset.index[4]]

        return "Following are the Recommended Questions----->" + b + c + d + e + f


In [19]:
def speech_to_text():
    
    r = sr.Recognizer() 

    try:

            with sr.Microphone() as source2:

                r.adjust_for_ambient_noise(source2, duration=0.2)

                print("Start Talking : \n")

                audio2 = r.listen(source2)

                print("Recognizing : ......\n")
                MyText = r.recognize_google(audio2)
                MyText = MyText.lower()

                print("\nDid you say :   "+MyText)
                
                return MyText

    except sr.RequestError as e:
        print("Could not request results; {0}".format(e))

    except sr.UnknownValueError:
        print("unknown error occured")
    

In [20]:
#trial
#How does my credit score impact the cost of my loan?




get_response2("personal loan interest rate")

[21, 18, 0, 17, 9]


'Following are the Recommended Questions----->(1)What is the lowest interest rate available on personal loans?(2)How does my income determine my interest rate?(3)How do my employment history and experience play a role in determining the rate of interest?(4)How does my credit score impact the cost of my loan?(5)How does my current debt level determine my interest rate?'

In [21]:
get_response("If I have a low credit score, can I still get a loan?")

'Yes, you may be able to get a low rate. You can try and get a good rate by getting a co-applicant with good credit to apply along with you. Another way is to get a guarantor with good credit to back you up.   '

In [22]:
#normal chat based

print("Welcome to finbot, you can ask me questions related to banking services!\n\n\n")
while(1):
  
    user_input = input("")
    if user_input!="0":
        print("\n\n\n"+get_response(user_input))
        
        print("\n\n\nWould you like to continue talking to me. If no, please press 0 to exit")
    else:
        print("\n\n\nThank you for using finbot")
        break
        
    

        
        
        

Welcome to finbot, you can ask me questions related to banking services!



Advantages of Neft



NEFT offers the following advantages for funds transfer or receipt: 
(i) Round the clock availability on all days of the year. 
(ii) Near-real-time funds transfer to the beneficiary account and settlement in a secure manner. 
(iii) Pan-India coverage through large network of branches of all types of banks. 
(iv) Positive confirmation to the remitter by SMS / e-mail on credit to beneficiary account. 
(v) Penal interest provision for delay in credit or return of transactions. 
(vi) No levy of charges by RBI from banks. No charges to savings bank account customers for online NEFT transactions. 
(vii) Besides funds transfer, NEFT system can be used for a variety of transactions including payment of credit card dues to the card issuing banks, payment of loan EMI, inward foreign exchange remittances, etc. 
(viii) Available for one-way funds transfers from India to Nepal.   



Would you like to 

In [ ]:
#Speech based 

print("Welcome to finbot, you can ask me questions related to banking services!\n\n\n")
while(1):
  
    user_input = speech_to_text()
    if user_input!="0":
        print("\n\n\n"+get_response(user_input))
        
        print("\n\n\nWould you like to continue talking to me. If no, please say 0 to exit")
    else:
        print("\n\n\nThank you for using finbot")
        break
        
    


Welcome to finbot, you can ask me questions related to banking services!



Start Talking : 



In [25]:
# Testing speech to text
print("Start talking")

user_input = speech_to_text()

print("\n\n\n"+get_response(user_input))

Start talking
Start Talking : 

Recognizing : ......


Did you say :   advantages of rtgs



RTGS offers many advantages over the other modes of funds transfer: 
(1) It is a safe and secure system for funds transfer. 
(2) RTGS transactions / transfers have no amount cap. 
(3) The system is available on all days on 24x7x365 basis. 
(4) There is real time transfer of funds to the beneficiary account. 
(5) The remitter need not use a physical cheque or a demand draft. 
(6) The beneficiary need not visit a bank branch for depositing the paper instruments. 
(7) The beneficiary need not be apprehensive about loss / theft of physical instruments or the likelihood of fraudulent encashment thereof. 
(8) Remitter can initiate the remittances from his / her home / place of work using internet banking, if his / her bank offers such service. 
(9) The transaction charges have been capped by RBI. The transaction has legal backing.   
